In [7]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
from concurrent.futures import ThreadPoolExecutor
import os

In [2]:
def create_db_connection(host_name, user_name, user_password, db_name):

    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        return connection
    except Error as err:
        print(f"Error: '{err}'")
        return None

In [3]:
def execute_batch_insert_with_offset(host, username, password, database, query, data, offset, batch_size):
    """
    """
    connection = create_db_connection(host, username, password, database)
    if connection:
        cursor = connection.cursor()
        try:
            # Calculate the actual batch based on offset and batch_size
            batch_data = data[offset:offset + batch_size]
            cursor.executemany(query, batch_data)
            connection.commit()
            print(f"Batch insert successful for offset {offset}")
        except Error as err:
            print(f"Error: '{err}'")
        finally:
            cursor.close()
            connection.close()

In [4]:
# Database Configuration from env variables
host = os.environ.get('DB_HOST')                    # Database host
username = os.environ.get('DB_USER')                # Database user
password = os.environ.get('DB_PASSWORD')            # Database password
database = 'Airline'   

In [5]:
cols = ["ActualElapsedTime","AirTime","ArrDelay","ArrTime","CRSArrTime","CRSDepTime","CRSElapsedTime","Cancelled","CarrierDelay","DayOfWeek","DayofMonth","DepDelay","DepTime","Dest","Distance","Diverted","FlightNum","LateAircraftDelay","Month","NASDelay","Origin","TailNum","TaxiIn","TaxiOut","UniqueCarrier","WeatherDelay","Year"]
len(cols)

27

In [6]:
# File Path Configuration
file_path = r'airline.csv'

In [7]:
dt = pd.DataFrame()

# Read data from CSV file
chunks = pd.read_csv(file_path, chunksize=5000, encoding='latin1', usecols=cols)




In [8]:
for idx, chunk in enumerate(chunks):
    if idx <= 1000:
        # Remove rows where any column has null value
        chunk = chunk.dropna()

        # Remove rows where any column contains 'NA' as a string
        chunk = chunk[~chunk.applymap(lambda x: x == 'NA').any(axis=1)]

        # Check if chunk still has rows
        if not chunk.empty:
            # If chunk is not empty, concatenate it with dt
            dt = pd.concat([dt, chunk], ignore_index=True)
        else:
            # If chunk is empty, continue to the next chunk
            # throw an error if there are no more chunks to process
            if idx == 0:
                raise ValueError('Dataframe contains no rows!')
            else:
                continue
    else:
        break


In [11]:
dt.shape

(1358066, 27)

In [13]:
dt.head

<bound method NDFrame.head of          ActualElapsedTime  AirTime  ArrDelay  ArrTime  CRSArrTime  \
0                     45.0     29.0       2.0   1120.0        1118   
1                    100.0     81.0      -2.0   1220.0        1222   
2                     33.0     22.0      -3.0   1916.0        1919   
3                     57.0     31.0      -9.0   1659.0        1708   
4                     95.0     74.0     136.0   2343.0        2127   
...                    ...      ...       ...      ...         ...   
1358061              150.0    135.0     228.0    115.0        2127   
1358062               71.0     55.0      -9.0   1734.0        1743   
1358063               65.0     52.0      -5.0   1035.0        1040   
1358064               96.0     77.0     -11.0   1419.0        1430   
1358065              198.0    166.0      13.0   1419.0        1406   

         CRSDepTime  CRSElapsedTime  Cancelled  CarrierDelay  DayOfWeek  ...  \
0              1030            48.0          0   

In [16]:
dt.to_csv('cleanAirline.csv', index=False)